# Databse Refactor
This notebook is used to refactor the CSV cleaned files into a database. The new database will use a different schema than the original CSV files to make it easier to query and analyze the data. The new schema will have the following tables:

Import requirements

In [118]:
import os
import random
import pandas as pd

In [78]:
# Read the csv
ubicaciones = pd.read_csv('../databases/cleaned-data/ubicaciones_cleaned.csv')
ubicaciones.head()

,ID_UBICACIO,DS_UBICACIO,ID_EDIFICI,CAPACIDAD
0,Q1/1003,aula q1/1003 (dues portes),Q,70
1,Q3/0013,aula q3/0013,Q,33
2,Q4/1013,aula q4/1013 (dues portes),Q,69
3,Q1/0007,aula d'informàtica a (2 portes,Q,68
4,Q6/2008,laboratori,Q,20


In [79]:
# Read the calendar group csv
calendar_group = pd.read_csv('../databases/cleaned-data/calendario_grupos_merged.csv')
calendar_group.head()

,ID_GRUPO,ID_FECHA_GRUPO,ID_HORA_INICIO,ID_HORA_FIN,ID_CURSO_ACADEMICO,ID_ASIGNATURA,ID_TIPO_DOCENCIA,ID_COD_GRUPO,ID_PERIODO_DOCENTE,IND_ALUMNOS_GRUPO_PREV,IND_ALUMNOS_GRUPO_REAL,IND_HORAS_PREVISTAS
0,2024-0-115-102708-54-311,2025-07-04,930,1030,2024,102708,54,311,1,45,50,6.0
1,2024-0-115-102764-54-472,2025-07-04,1700,1900,2024,102764,54,472,1,37,44,50.0
2,2024-0-115-104554-54-1,2025-07-03,1700,1900,2024,104554,54,1,1,23,21,12.0
3,2024-0-115-102764-54-472,2025-06-27,1700,1900,2024,102764,54,472,1,37,44,50.0
4,2024-0-115-102708-54-311,2025-06-27,930,1030,2024,102708,54,311,1,45,50,6.0


In [100]:
# Read the caracteristicas recursos csv
caracteristicas_recursos = pd.read_csv('../databases/cleaned-data/caracteristicas_recursos_count.csv')
caracteristicas_recursos.head()

,DS_CARACTERISTICA,Count
0,Piano,1
1,Escaner diapositives,1
2,Capacitat,6
3,Traducció simultània (130 CAD),8
4,Videoconferència,8


In [90]:
# Get all the distinct values of the column "ID_EDIFICI"
edficis = ubicaciones['ID_EDIFICI'].unique()

# Create a new building csv file with the column "id", which is the same as "ID_EDIFICI"
buildings = pd.DataFrame(edficis, columns=['id'])

# Add "Edifici " to the column "id" and save it as a "name" column
buildings['name'] = 'Edifici ' + buildings['id'].astype(str)

# Sort the values by the column "id"
buildings = buildings.sort_values(by='id')

# Change the index to the column "id"
buildings = buildings.set_index('id')

# Remove rows that are not Q
buildings = buildings.drop(buildings[buildings.index != 'Q'].index)

buildings

,name
id,
Q,Edifici Q


In [93]:
# Create a new "spaces" csv file with the columns "id", "name" and "capacity"
spaces = pd.DataFrame(columns=['id', 'name'])

# Add the values "ID_UBICACIO", "DS_UBICACIO" and "CAPACIDAD" to the columns "id", "name" and "capacity" respectively
spaces['id'] = ubicaciones['ID_UBICACIO']
spaces['building_id'] = ubicaciones['ID_EDIFICI']
spaces['name'] = ubicaciones['DS_UBICACIO']
spaces['capacity'] = ubicaciones['CAPACIDAD']

# Add "is_active" to the columns
spaces["is_active"] = True

# Sort the values by the column "id"
spaces = spaces.sort_values(by='id')

# Remove the rows that are not Q
spaces = spaces.drop(spaces[spaces['building_id'] != 'Q'].index)

# Change the index to the column "id"
spaces = spaces.set_index('id')

# Remove the duplicates
spaces = spaces.drop_duplicates()

spaces

,name,building_id,capacity,is_active
id,,,,
Q1/0001,pas. aula inf.+ sala graus,Q,27,True
Q1/0007,aula d'informàtica a (2 portes,Q,68,True
Q1/0019,aula d'informàtica b,Q,68,True
Q1/1003,aula q1/1003 (dues portes),Q,70,True
Q1/1007,aula q1/1007 (dues portes),Q,68,True
Q1/1011,aula q1/1011 (tres portes),Q,103,True
Q2/0001,pas.lab.int. + sala d'estudis,Q,27,True
Q2/0007,lab. integrat informàtica 1,Q,21,True
Q2/0011,lab. integrat informàtica 2,Q,21,True


In [94]:
# Get all the values of the column "ID_GRUPO"
group_ids = calendar_group['ID_GRUPO']

# From \d+ - \d+ - \d+ ..., extract the third number (ex: 2024-0-115-102708-54-311 -> 115)
building_ids = group_ids.str.extract(r'[\d]+-[\d]+-([\d]+)-[\d]+-[\d]+-[\d]+')

# Add "ID_EDIFICI" as a column
calendar_group['ID_EDIFICI'] = building_ids

# Get all the distinct values of the column "ID_EDIFICI"
building_ids = building_ids[0].unique()

# Set the relationship between the building id and the real id in a map
building_relations = {}
building_relations['115'] = "Q"

# Create a new "subject" csv file with the columns "id", "building_id", "name" and "period"
subjects = pd.DataFrame(columns=['id', 'building_id', 'name', 'period'])

# Add the values "id", "building_id", "name" and "period" to the columns
subjects['id'] = calendar_group['ID_ASIGNATURA']
subjects['building_id'] = calendar_group['ID_EDIFICI']
subjects['name'] = 'Asignatura ' + calendar_group['ID_ASIGNATURA'].astype(str)
subjects['period'] = calendar_group['ID_PERIODO_DOCENTE']

# Change the values of the column "building_id" to the real values (if not, delete the row)
subjects['building_id'] = subjects['building_id'].map(building_relations)
subjects = subjects.dropna()

# Remove the duplicates
subjects = subjects.drop_duplicates()

# Sort the values by the column "id"
subjects = subjects.sort_values(by='id')

# Change the index to the column "id"
subjects = subjects.set_index('id')

subjects

,building_id,name,period
id,,,
44728,Q,Asignatura 44728,1
44729,Q,Asignatura 44729,1
44730,Q,Asignatura 44730,1
44731,Q,Asignatura 44731,1
44732,Q,Asignatura 44732,1
...,...,...,...
106939,Q,Asignatura 106939,1
106940,Q,Asignatura 106940,1
106941,Q,Asignatura 106941,1


In [95]:
# Create a new "subject_group" csv file with the columns "id", "subject_id", "year" and "duration"
subject_groups = pd.DataFrame(columns=['id', 'subject_id', 'year', 'duration'])

# Add the values "id", "building_id", "name" and "period" to the columns
subject_groups['id'] = calendar_group['ID_COD_GRUPO']
subject_groups['subject_id'] = calendar_group['ID_ASIGNATURA']
subject_groups['year'] = calendar_group['ID_CURSO_ACADEMICO']

# Remove the duplicates
subject_groups = subject_groups.drop_duplicates()

# Sort the values by the column "id"
subject_groups = subject_groups.sort_values(by='id')

# Change the index to the column "id"
subject_groups = subject_groups.set_index('id')

subject_groups

,subject_id,year,duration
id,,,
1,44757,2024,NaN
1,101739,2024,NaN
1,101738,2024,NaN
1,44733,2024,NaN
1,44732,2024,NaN
...,...,...,...
812,104345,2024,NaN
813,104352,2024,NaN
813,104339,2024,NaN


In [96]:
calendar_group.head()

,ID_GRUPO,ID_FECHA_GRUPO,ID_HORA_INICIO,ID_HORA_FIN,ID_CURSO_ACADEMICO,ID_ASIGNATURA,ID_TIPO_DOCENCIA,ID_COD_GRUPO,ID_PERIODO_DOCENTE,IND_ALUMNOS_GRUPO_PREV,IND_ALUMNOS_GRUPO_REAL,IND_HORAS_PREVISTAS,ID_EDIFICI
0,2024-0-115-102708-54-311,2025-07-04,930,1030,2024,102708,54,311,1,45,50,6.0,115
1,2024-0-115-102764-54-472,2025-07-04,1700,1900,2024,102764,54,472,1,37,44,50.0,115
2,2024-0-115-104554-54-1,2025-07-03,1700,1900,2024,104554,54,1,1,23,21,12.0,115
3,2024-0-115-102764-54-472,2025-06-27,1700,1900,2024,102764,54,472,1,37,44,50.0,115
4,2024-0-115-102708-54-311,2025-06-27,930,1030,2024,102708,54,311,1,45,50,6.0,115


In [97]:
# Create a new "subject_group" csv file with the columns "id", "subject_group_id", "space_id", "day", "start", "end", "planned_capacity", "real_capacity", "is_morning"
subject_group_sessions = pd.DataFrame(columns=['id', 'subject_group_id', 'space_id', 'day', 'start', 'end', 'planned_capacity', 'real_capacity', 'is_morning'])

# Add the values "id", "subject_group_id", "year", "planned_capacity", "real_capacity", "hours" and "is_morning" to the columns
subject_group_sessions['subject_group_id'] = calendar_group['ID_COD_GRUPO']
subject_group_sessions['day'] = calendar_group['ID_FECHA_GRUPO']
subject_group_sessions['start'] = calendar_group['ID_HORA_INICIO']
subject_group_sessions['end'] = calendar_group['ID_HORA_FIN']
subject_group_sessions['planned_capacity'] = calendar_group['IND_ALUMNOS_GRUPO_PREV']
subject_group_sessions['real_capacity'] = calendar_group['IND_ALUMNOS_GRUPO_REAL']
subject_group_sessions['is_morning'] = calendar_group['ID_HORA_INICIO'].astype(int) < 1430

# Convert day to a date format to extract the day of the week (0-7)
subject_group_sessions['day'] = pd.to_datetime(subject_group_sessions['day'], format='%Y-%m-%d')
subject_group_sessions['day'] = subject_group_sessions['day'].dt.dayofweek

# Convert the start and end to a time format (from 930 to 9:30)
subject_group_sessions['start'] = subject_group_sessions['start'].astype(str)
subject_group_sessions['end'] = subject_group_sessions['end'].astype(str)
subject_group_sessions['start'] = subject_group_sessions['start'].str.zfill(4)
subject_group_sessions['end'] = subject_group_sessions['end'].str.zfill(4)
subject_group_sessions['start'] = subject_group_sessions['start'].str[:2] + ':' + subject_group_sessions['start'].str[2:]
subject_group_sessions['end'] = subject_group_sessions['end'].str[:2] + ':' + subject_group_sessions['end'].str[2:]

# Create duration (in seconds) for each session
subject_group_sessions['start_date'] = pd.to_datetime(subject_group_sessions['start'], format='%H:%M')
subject_group_sessions['end_date'] = pd.to_datetime(subject_group_sessions['end'], format='%H:%M')
subject_group_sessions['duration'] = (subject_group_sessions['end_date'] - subject_group_sessions['start_date']).dt.total_seconds().astype(int)

# Remove the columns "start_date" and "end_date"
subject_group_sessions = subject_group_sessions.drop(columns=['start_date', 'end_date'])

# Remove the duplicates
subject_group_sessions = subject_group_sessions.drop_duplicates()

# Sort the values by the column "subject_group_id", "day", "start" and "end"
subject_group_sessions = subject_group_sessions.sort_values(by=['subject_group_id', 'day', 'start', 'end'])

# Add "id" to the columns (autoincremental)
subject_group_sessions['id'] = range(1, len(subject_group_sessions) + 1)

# Change the index to the column "id"
subject_group_sessions = subject_group_sessions.set_index('id')

subject_group_sessions

,subject_group_id,space_id,day,start,end,planned_capacity,real_capacity,is_morning,duration
id,,,,,,,,,
1,1,NaN,0,09:30,11:30,17,18,True,7200
2,1,NaN,0,11:30,12:30,17,18,True,3600
3,1,NaN,0,12:30,14:00,17,18,True,5400
4,1,NaN,0,12:30,14:30,20,16,True,7200
5,1,NaN,0,15:00,17:00,74,70,False,7200
...,...,...,...,...,...,...,...,...,...
1306,812,NaN,4,13:30,14:30,30,34,True,3600
1307,813,NaN,0,15:00,17:00,20,23,False,7200
1308,813,NaN,2,08:30,11:30,20,21,True,10800


In [98]:
# Get the sum of the column "duration" group by "subject_group_id"
subject_group_sessions_duration = subject_group_sessions.groupby('subject_group_id')['duration'].sum()

# Add "duration" to the columns of "subject_groups"
subject_groups['duration'] = subject_group_sessions_duration

subject_groups

,subject_id,year,duration
id,,,
1,44757,2024,369000
1,101739,2024,369000
1,101738,2024,369000
1,44733,2024,369000
1,44732,2024,369000
...,...,...,...
812,104345,2024,79200
813,104352,2024,28800
813,104339,2024,28800


In [120]:
# Create a new "feature" csv file with the columns "id", "name" and "count"
features = pd.DataFrame(columns=['id', 'name', 'count'])

# Add the values "id" and "name" to the columns
features['name'] = caracteristicas_recursos['DS_CARACTERISTICA']
features['count'] = caracteristicas_recursos['Count']

# Add "id" to the columns (autoincremental)
features['id'] = range(1, len(features) + 1)

# Change the index to the column "id"
features = features.set_index('id')

features

,name,count
id,,
1,Piano,1
2,Escaner diapositives,1
3,Capacitat,6
4,Traducció simultània (130 CAD),8
5,Videoconferència,8
...,...,...
58,Ordinador,135
59,Connexió xarxa,135
60,Capacitat docència,137


In [135]:
# Convert the feature to a list (dictionary with id, name and count)
features_list = features.to_dict('records')
for i, feature in enumerate(features_list):
  feature['id'] = i + 1

# Create a new "space_feature" csv file with the columns "space_id", "feature_id"
space_features = pd.DataFrame(columns=['id', 'space_id', 'feature_id'])

# Assign random features to the spaces and decrease the count of the feature
for index, row in spaces.iterrows():
    for feature in features_list:
        if feature['count'] <= 0:
            continue
	        
        # Get a random space (use always the same seed)
        random.seed(index)
        space_id = random.choice(spaces.index)
	      
        # Add the space and the feature to the csv
        sf_row = pd.DataFrame([[space_id, feature['id']]], columns=['space_id', 'feature_id'])
        
        # Append the row to the csv
        space_features = pd.concat([space_features, sf_row])
	    
        # Decrease the count of the feature
        feature['count'] -= 1
	    
        # Break the loop if the count is 0
        if feature['count'] <= 0:
            break

# Add "id" to the columns (autoincremental)
space_features['id'] = range(1, len(space_features) + 1)

# Change the index to the column "id"
space_features = space_features.set_index('id')

space_features

,space_id,feature_id
id,,
1,QC/2011,1
2,Q2/0017,2
3,Q3/0013,3
4,Q3/0013,4
5,Q3/0013,5
...,...,...
1823,Q6/2004,59
1824,Q6/2004,60
1825,Q6/2004,61


In [138]:
# Copy the features dataframe
features_copy = features.copy()

# Change the "count" field to theis 25% (to int)
features_copy['count'] = (features_copy['count'] * 0.1).astype(int)

features_copy

,name,count
id,,
1,Piano,0
2,Escaner diapositives,0
3,Capacitat,0
4,Traducció simultània (130 CAD),0
5,Videoconferència,0
...,...,...
58,Ordinador,13
59,Connexió xarxa,13
60,Capacitat docència,13


In [139]:
# Convert the feature to a list (dictionary with id, name and count)
features_list = features_copy.to_dict('records')
for i, feature in enumerate(features_list):
  feature['id'] = i + 1
  
# Create a new "subject_group_space_feature" csv file with the columns "id", "subject_group_session_id" and "feature_id"
subject_group_space_features = pd.DataFrame(columns=['id', 'subject_group_session_id', 'feature_id'])

# Assign random features to the spaces and decrease the count of the feature
for index, row in subject_group_sessions.iterrows():
    for feature in features_list:
        if feature['count'] <= 0:
            continue
          
        # Get a random space (use always the same seed)
        random.seed(index)
        feature_id = random.choice(features_copy.index)
        
        # Add the space and the feature to the csv
        sf_row = pd.DataFrame([[index, feature['id']]], columns=['subject_group_session_id', 'feature_id'])
        
        # Append the row to the csv
        subject_group_space_features = pd.concat([subject_group_space_features, sf_row])
      
        # Decrease the count of the feature
        feature['count'] -= 1
      
        # Break the loop if the count is 0
        if feature['count'] <= 0:
            break
            
# Add "id" to the columns (autoincremental)
subject_group_space_features['id'] = range(1, len(subject_group_space_features) + 1)

# Change the index to the column "id"
subject_group_space_features = subject_group_space_features.set_index('id')

subject_group_space_features

,subject_group_session_id,feature_id
id,,
1,1,12
2,2,13
3,3,14
4,4,15
5,5,16
...,...,...
359,64,61
360,64,62
361,65,61


In [140]:
# Create the directory if it does not exist
if not os.path.exists('../databases/data'):
    os.makedirs('../databases/data')
    
# Save the csv files
buildings.to_csv('../databases/data/buildings.csv')
spaces.to_csv('../databases/data/spaces.csv')
subjects.to_csv('../databases/data/subjects.csv')
subject_groups.to_csv('../databases/data/subject_groups.csv')
subject_group_sessions.to_csv('../databases/data/subject_group_sessions.csv')
features.to_csv('../databases/data/features.csv')
space_features.to_csv('../databases/data/space_features.csv')
subject_group_space_features.to_csv('../databases/data/subject_group_space_features.csv')
